In [ ]:
# default_exp objects

In [ ]:
#hide 
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
#export
from rtma.imports import *
from rtma.core import *
from rtma.signal import *
from rtma.stft import *

# Objects

## Signal

In [ ]:
class Signal:
    def __init__(self, filename: str) -> None:
        self.filename = filename        
        self.x, self.sample_rate = load_audio(filename)
    
    @property
    def n_samples(self) -> int:
        return self.x.shape[-1]
    
    @property
    def duration(self) -> float:
        return self.n_samples / self.sample_rate
        
    def __len__(self) -> int:
        return self.n_samples
    
    def play(self) -> None:
        play_audio(self.x, self.sample_rate)
        
    def plot(self) -> None:
        plot(self.x)
        
    def __str__(self) -> str:
        return f'Signal("{self.filename}")'
    
    def __repr__(self) -> str:
        return str(self)

In [ ]:
s = Signal("data/E_octaves_both.wav")
s

In [ ]:
s.play()
s.plot()

In [ ]:
s.duration, s.n_samples, len(s)

In [ ]:
fclass Frame:
    def __init__(self, x: np.ndarray):
        self.x = x
    
    def plot(self):
        plot(self.x)

In [ ]:
class Analysis:
    def __init__(self, 
                 signal: Signal,
                 frame_size: int, 
                 hop_size: int) -> None:
        self.signal = signal
        self.frame_size = frame_size
        self.hop_size = hop_size
    
    def __repr__(self) -> str:
        return f'Analysis({self.signal}, frame_size={self.frame_size}, hop_size={self.hop_size})'
    
    @property
    def overlap(self) -> int:
        return self.frame_size - self.hop_size
    
    @property
    def frames(self):
        x_pad = np.pad(self.signal.x, (self.overlap, 0), mode='constant')
        i = 0
        while i < x_pad.size:
            x_frame = x_pad[i:i+self.frame_size]
            if x_frame.size < self.frame_size:
                x_frame = np.pad(x_frame, (0, self.frame_size-x_frame.size), mode='constant')
            yield Frame(x_frame)
            i += self.hop_size

In [ ]:
a = Analysis(s, frame_size=601, hop_size=150)
frames = list(a.frames)
len(frames), a

In [ ]:
class SpectralAnalysis(Analysis):
    def __init__(self,
                 *args,
                 fft_size: int,
                 window_name: str,
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.fft_size = fft_size
        self.window_name = window_name
        self.window = get_cola_window(window_name, self.frame_size, self.hop_size)
        print(self.window.max())
    
    @property
    def frames(self):
        for frame in super().frames:
            yield Frame(frame.x * self.window)

In [ ]:
sa = SpectralAnalysis(s, 
                      fft_size=2048,
                      window_name='hamming',
                      frame_size=601, 
                      hop_size=150)